In [1]:
import os
import pandas as pd
import numpy as np
import time
from sklearn.metrics import log_loss

In [2]:
os.chdir('/aichallenge')

In [3]:
import time
t1=time.time()
train = pd.read_table('challengeData.tsv')
test = pd.read_table('scoring_set.tsv')
train_imputed = pd.read_csv('train_imputed.csv')
test_imputed = pd.read_csv('test_imputed.csv')
t2=time.time()
print(t2-t1)

/usr/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (6,16,17,26,40,41,49,53,97,98,99,100,104,107,109,112,127,128,129,130,220,221,222) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (6,16,17,26,40,41,49,53,54,55,100,104,107,109,112,127,128,129,130,153,171,220,221,222,225,226,227) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


215.79182720184326


In [4]:
train.shape

(250000, 237)

In [5]:
train_imputed.isnull().any().any()

False

In [6]:
test_imputed.shape

(1888195, 42)

In [7]:
train_imputed.shape

(226140, 42)

In [8]:
train_imputed = train_imputed.drop('Unnamed: 0',axis=1)
test_imputed = test_imputed.drop('Unnamed: 0',axis=1)

In [9]:
#remove data set name from columns names to make it uniform
t1=time.time()
columns=[]
for col in train.columns:
    columns.append(col.replace('challenge_data.',''))
train.columns=columns
columns=[]
for col in test.columns:
    columns.append(col.replace('scoring_set.',''))
test.columns=columns
t2=time.time()
print(t2-t1)

0.00121307373046875


In [10]:
train['renewed_yorn'].isnull().sum()

23860

In [11]:
#remove the rows where 'challenge_data.renewed_yorn' is null in training data
t1=time.time()
train=train[train['renewed_yorn'].notnull()]
t2=time.time()
print(t2-t1)

0.6240081787109375


In [12]:
#remove the 'challenge_data.renewed_yorn' from training and test data and move it to a seperate variable
t1=time.time()
train_y=train['renewed_yorn']
train = train.drop(['renewed_yorn','zip_code'],axis=1)
test_y=test['renewed_yorn']
test = test.drop(['renewed_yorn','zip_code'],axis=1)
t2=time.time()
print(t2-t1)

4.648179531097412


In [13]:
#remove columns which have many null values
t1=time.time()
percent = (test.isnull().sum()/test.isnull().count())
test.columns[percent>0.5]
#We would ignore all these 102 columns so we are left with 135 columns
len(test.columns[percent>0.5])
#all_clean_columns=train.columns[percent==0]
#train_clean=train[all_clean_columns]
unclean_columns=test.columns[percent>0.5]
clean_columns=test.columns[percent==0]
train = train.drop(unclean_columns,axis=1)
test = test.drop(unclean_columns,axis=1)
t2=time.time()
print(t2-t1)

52.704150915145874


In [14]:
len(unclean_columns)

36

In [15]:
len(clean_columns)

133

In [16]:
t1=time.time()
percent = (test.isnull().sum()/test.isnull().count())
target_columns=test.columns[percent>0]
train_missing=train[target_columns]
test_missing=test[target_columns]
train = train.drop(target_columns,axis=1)
test = test.drop(target_columns,axis=1)

In [17]:
len(target_columns)

66

In [18]:
#Remove columns which have too many categories and is string type
t1=time.time()
messy_columns = []
for col in target_columns:
    if(train_missing[col].dtype=='O' and len(train_missing[col].unique())>=10):
        messy_columns.append(col)
t2=time.time()
print(t2-t1)

1.7744455337524414


In [19]:
# 56 messy columns are removed
len(messy_columns)

25

In [20]:
len(train_missing.columns)

66

In [21]:
t1=time.time()
train_missing=train_missing.drop(messy_columns,axis=1)
test_missing=test_missing.drop(messy_columns,axis=1)
t2=time.time()
print(t2-t1)

0.9078669548034668


In [22]:
#Remove columns which have too many categories and is string type
t1=time.time()
messy_columns = []
for col in train.columns:
    if(train[col].dtype=='O' and len(train[col].unique())>=10):
        messy_columns.append(col)
t2=time.time()
print(t2-t1)

1.7493765354156494


In [23]:
len(messy_columns)

61

In [24]:
t1=time.time()
train=train.drop(messy_columns,axis=1)
test=test.drop(messy_columns,axis=1)
t2=time.time()
print(t2-t1)

1.8785974979400635


In [25]:
train.shape

(226140, 72)

In [26]:
train_missing.shape

(226140, 41)

In [27]:
train_imputed.shape

(226140, 41)

In [28]:
train_imputed.isnull().any().any()

False

In [29]:
train.isnull().any().any()

False

In [30]:
train_copy=train

In [33]:
#Remove the primary key column for data fitting
t1=time.time()
train = train.drop('innovation_challenge_key',axis=1)
test_ids=test['innovation_challenge_key']
test = test.drop('innovation_challenge_key',axis=1)
t2=time.time()
print(t2-t1)

KeyError: 'innovation_challenge_key'

In [32]:
train=train_copy

In [34]:
train.isnull().any().any()

False

In [35]:
train=train.reset_index()

In [36]:
train = train.drop(['index','innovation_challenge_key'],axis=1)

ValueError: labels ['innovation_challenge_key'] not contained in axis

In [37]:
for col in train_imputed.columns:
    train[col]=train_imputed[col]
    test[col]=test_imputed[col]

In [38]:
train[col].shape

(226140,)

In [45]:
train.shape

(226140, 112)

In [43]:
train = train.drop('index',axis=1)

In [44]:
test.shape

(1888195, 112)

In [46]:
train.isnull().any().any()

False

In [47]:
#Number of unique values in each columns
t1=time.time()
catcols=[]
scalcols=[]
for col in train.columns:
    if (train[col].dtype !='object'):
        if(len(train[col].unique())<=10):
            catcols.append(col)
        else:
            scalcols.append(col)
t2=time.time()
print(t2-t1)

0.05004405975341797


In [48]:
#handle catcols
t1=time.time()
for col in catcols:
    train[col]=train[col].astype('category')
    test[col]=test[col].astype('category')
t2=time.time()
print(t2-t1)

0.25840210914611816


In [49]:
train.isnull().any().sum()

0

In [50]:
test.isnull().any().any()

False

In [51]:
#Normalize integer data
t1=time.time()
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler = scaler.fit(train[scalcols])
train[scalcols] = scaler.transform(train[scalcols])
scaler = scaler.fit(test[scalcols])
test[scalcols] = scaler.transform(test[scalcols])
t2=time.time()
print(t2-t1)

3.6095423698425293


In [52]:
train[scalcols].describe()

,contract_line_net_usd_amount,product_net_price,sales_node_renewal_rate,partner_renewal_rate,product_renewal_rate,service_sales_node_installed_base_sales_node_renewal_rate,service_partner_installed_base_partner_renewal_rate
count,2.261400e+05,2.261400e+05,2.261400e+05,2.261400e+05,2.261400e+05,2.261400e+05,2.261400e+05
mean,1.005455e-17,1.206546e-17,1.754520e-16,-1.759547e-18,4.524549e-18,-3.016366e-17,-2.686451e-18
std,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00
min,-1.252316e-01,-2.321716e-01,-2.737838e+00,-1.742868e-01,-7.790595e-02,-1.128741e+00,-5.226844e-02
25%,-1.252316e-01,-2.321716e-01,-6.842019e-01,-1.053205e-01,-7.719748e-02,-2.906674e-01,-2.610574e-02
50%,-1.252316e-01,-2.321716e-01,-7.744580e-02,-4.824491e-02,-7.436361e-02,-1.738249e-02,-1.613900e-02
75%,-1.252316e-01,-1.584198e-01,2.959425e-01,6.452532e-03,-6.515353e-02,1.283695e-01,-1.188889e-03
max,7.774224e+01,6.648806e+01,4.118197e+01,9.915151e+01,1.359952e+02,4.662324e+01,1.670925e+02


In [53]:
train[catcols].describe()

,product_transaction_type,contract_line_reaction_time_code,sales_hierarchy_level,service_sales_node_base_sales_hierarchy_level
count,226140,226140,226140,226140
unique,3,3,6,6
top,10002,-1,6,6
freq,202522,225296,199960,224170


In [54]:
for col in train.columns:
    print(col+' '+str(len(train[col].unique())))

minor_line_yorn 2
instance_status 1
product_transaction_type 3
installed_product_type 2
installed_product_component_yorn 2
component_type 4
scdc_product_yorn 1
service_classification 2
contract_line_multi_year_discount_yorn 2
contract_line_route_to_market_code 3
contract_line_sales_motion_code 1
default_multi_year_discount_yorn 2
low_dollar_contract_yorn 2
contract_line_source_business_process_name 2
contract_line_repair_service_order_level 2
contract_line_reaction_time_code 3
product_route_to_market 5
service_route_to_market_code 5
auto_quote_yorn 2
hardware_yorn 2
software_yorn 2
acw_yorn 2
enterprise_license_agreement_yorn 2
software_opportunity_type 3
sales_hierarchy_level 6
united_states_yorn 2
level_1_name 5
service_sales_node_base_sales_hierarchy_level 6
service_sales_node_base_scms_name 6
service_sales_node_base_united_states_yorn 2
service_sales_node_base_country_type 3
service_sales_node_base_level_1_name 6
item_type 5
manufacturing_plant 2
software_stack 4
monetization_type 

In [55]:
cat1_cols = train.columns[0:28]
cat2_cols = train.columns[28:56]
cat3_cols = train.columns[56:84]
cat4_cols = train.columns[84:112]

In [58]:
train_cat1 = train[cat1_cols]
train_cat2 = train[cat2_cols]
train_cat3 = train[cat3_cols]
train_cat4 = train[cat4_cols]
test_cat1 = test[cat1_cols]
test_cat2 = test[cat2_cols]
test_cat3 = test[cat3_cols]
test_cat4 = test[cat4_cols]

In [59]:
#convert data to one hot encoding to handle categorical values
t1=time.time()
train_objs_num = len(train_cat1)
dataset = pd.concat(objs=[train_cat1, test_cat1], axis=0)
dataset = pd.get_dummies(dataset)
train_cat1 = dataset[:train_objs_num]
test_cat1 = dataset[train_objs_num:]
t2=time.time()
print(t2-t1)

5.087857723236084


In [60]:
#convert data to one hot encoding to handle categorical values
t1=time.time()
train_objs_num = len(train_cat2)
dataset = pd.concat(objs=[train_cat2, test_cat2], axis=0)
dataset = pd.get_dummies(dataset)
train_cat2 = dataset[:train_objs_num]
test_cat2 = dataset[train_objs_num:]
t2=time.time()
print(t2-t1)

4.220139026641846


In [61]:
#convert data to one hot encoding to handle categorical values
t1=time.time()
train_objs_num = len(train_cat3)
dataset = pd.concat(objs=[train_cat3, test_cat3], axis=0)
dataset = pd.get_dummies(dataset)
train_cat3 = dataset[:train_objs_num]
test_cat3 = dataset[train_objs_num:]
t2=time.time()
print(t2-t1)

4.799879789352417


In [62]:
#convert data to one hot encoding to handle categorical values
t1=time.time()
train_objs_num = len(train_cat4)
dataset = pd.concat(objs=[train_cat4, test_cat4], axis=0)
dataset = pd.get_dummies(dataset)
train_cat4 = dataset[:train_objs_num]
test_cat4 = dataset[train_objs_num:]
t2=time.time()
print(t2-t1)

3.7328031063079834


In [63]:
train_cat1.shape

(226140, 72)

In [64]:
train_cat2.shape

(226140, 84)

In [65]:
train_cat3.shape

(226140, 65)

In [66]:
train_cat4.shape

(226140, 61)

In [67]:
train_cat1.isnull().any().any()

False

In [68]:
test_cat1.isnull().any().any()

False

In [69]:
#split the data between train and validation set
t1=time.time()
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(train_cat1,train_y,test_size=0.33, random_state=42)
t2=time.time()
print(t2-t1)

0.47780752182006836


In [70]:
X_train.shape

(151513, 72)

In [71]:
import gc
del dataset
gc.collect()

0

In [72]:
def XGB_CV(
          max_depth,
          gamma,
          min_child_weight,
          max_delta_step,
          subsample,
          colsample_bytree
         ):
    global AUCbest
    global ITERbest

#
# Define all XGboost parameters
#

    paramt = {
        'booster' : 'gbtree',
        'max_depth' : int(max_depth),
        'gamma' : gamma,
        'eta' : 0.01,
        'objective' : 'binary:logistic',
        'nthread' : 4,
        'silent' : True,
        'eval_metric': 'logloss',
        'subsample' : max(min(subsample, 1), 0),
        'colsample_bytree' : max(min(colsample_bytree, 1), 0),
        'min_child_weight' : min_child_weight,
        'max_delta_step' : int(max_delta_step),
        'seed' : 1001
        }

    folds = 5
    cv_score = 0

    xgbc = xgb.cv(
                    paramt,
                    dtrain,
                    num_boost_round = 500,
                    stratified = True,
                    nfold = folds,
#                    verbose_eval = 10,
                    early_stopping_rounds = 20,
                    metrics = 'logloss',
                    show_stdv = True
               )

    val_score = xgbc['test-logloss-mean'].iloc[-1]
    train_score = xgbc['train-logloss-mean'].iloc[-1]
    print(' Stopped after %d iterations with train-auc = %f val-auc = %f ( diff = %f ) train-gini = %f val-gini = %f' % ( len(xgbc), train_score, val_score, (train_score - val_score), (train_score*2-1),
    (val_score*2-1)) )
    if ( val_score > AUCbest ):
        AUCbest = val_score
        ITERbest = len(xgbc)
    return (val_score*2) - 1

In [73]:
from bayes_opt import BayesianOptimization
XGB_BO = BayesianOptimization(XGB_CV, {
                                     'max_depth': (2, 12),
                                     'gamma': (0.001, 50.0),
                                     'min_child_weight': (0, 20),
                                     'max_delta_step': (0, 10),
                                     'subsample': (0.4, 1.0),
                                     'colsample_bytree' :(0.4, 1.0)
                                    })

In [76]:
t1=time.time()
import xgboost as xgb
import warnings
print('-'*130)
log_file = open('Porto-AUC-5fold-XGB-run-01-v1-full1.log', 'a')
AUCbest = -1.
ITERbest = 0
y_dtrain=pd.get_dummies(y_train)
dtrain = xgb.DMatrix(X_train, label = y_dtrain['Y'])
print('-'*130, file=log_file)
log_file.flush()

with warnings.catch_warnings():
    warnings.filterwarnings('ignore')
    XGB_BO.maximize(init_points=2, n_iter=5, acq='ei', xi=0.0)
t2=time.time()
print(t2-t1)

----------------------------------------------------------------------------------------------------------------------------------
Initialization
----------------------------------------------------------------------------------------------------------------------------------
 Step |   Time |      Value |   colsample_bytree |     gamma |   max_delta_step |   max_depth |   min_child_weight |   subsample | 
 Stopped after 500 iterations with train-auc = 0.573196 val-auc = 0.574420 ( diff = -0.001224 ) train-gini = 0.146391 val-gini = 0.148839
    1 | 03m06s |    0.14884 |             0.5801 |   14.8005 |           7.0240 |      7.6296 |             4.6199 |      0.9846 | 
 Stopped after 500 iterations with train-auc = 0.576014 val-auc = 0.576829 ( diff = -0.000815 ) train-gini = 0.152028 val-gini = 0.153658
    2 | 06m44s |    0.15366 |             0.9077 |   26.4332 |           3.8003 |      9.6833 |             0.8940 |      0.7076 | 
Bayesian Optimization
-----------------------------

In [77]:
XGB_BO.res['max']['max_val']

0.1797896000000001

In [78]:
model = []
model.append(XGB_BO.res['max']['max_params'])

In [102]:
X_train, X_valid, y_train, y_valid = train_test_split(train_cat1,train_y,test_size=0.33, random_state=42)

In [103]:
t1=time.time()
m_params = model[0]
#xgb1=xgb.XGBClassifier(max_depth=int(m_params['max_depth']),gamma=m_params['gamma'],colsample_bytree=m_params['colsample_bytree'],
#                       eval_metric='logloss',max_delta_step= m_params['max_delta_step'],min_child_weight=m_params['min_child_weight'],
#                      subsample= m_params['subsample'])
xgb1=xgb.XGBClassifier()
pred=xgb1.fit(X_train,y_train).predict_proba(X_valid)
pred0 = xgb1.predict_proba(test_cat1)
print(log_loss(y_valid,pred[:,1]))
t2=time.time()
t2-t1

0.579279515010252


29.409353733062744

In [104]:
X_train, X_valid, y_train, y_valid = train_test_split(train_cat2,train_y,test_size=0.33, random_state=42)

In [80]:
t1=time.time()
import xgboost as xgb
import warnings
print('-'*130)
log_file = open('Porto-AUC-5fold-XGB-run-01-v1-full1.log', 'a')
AUCbest = -1.
ITERbest = 0
y_dtrain=pd.get_dummies(y_train)
dtrain = xgb.DMatrix(X_train, label = y_dtrain['Y'])
print('-'*130, file=log_file)
log_file.flush()

with warnings.catch_warnings():
    warnings.filterwarnings('ignore')
    XGB_BO.maximize(init_points=2, n_iter=5, acq='ei', xi=0.0)
    
model.append(XGB_BO.res['max']['max_params'])

m_params = model[1]
xgb1=xgb.XGBClassifier(max_depth=int(m_params['max_depth']),gamma=m_params['gamma'],colsample_bytree=m_params['colsample_bytree'],
                       eval_metric='logloss',max_delta_step= m_params['max_delta_step'],min_child_weight=m_params['min_child_weight'],
                      subsample= m_params['subsample'])
pred=xgb1.fit(X_train,y_train).predict_proba(X_valid)
pred1 = xgb1.predict_proba(test_cat2)
print(log_loss(y_valid,pred[:,1]))

t2=time.time()
print(t2-t1)

----------------------------------------------------------------------------------------------------------------------------------
Bayesian Optimization
----------------------------------------------------------------------------------------------------------------------------------
 Step |   Time |      Value |   colsample_bytree |     gamma |   max_delta_step |   max_depth |   min_child_weight |   subsample | 
 Stopped after 500 iterations with train-auc = 0.594052 val-auc = 0.594160 ( diff = -0.000108 ) train-gini = 0.188104 val-gini = 0.188320
    8 | 02m11s |    0.18832 |             0.8056 |   49.8911 |           1.2655 |      2.1317 |             0.4858 |      0.5091 | 
 Stopped after 500 iterations with train-auc = 0.593786 val-auc = 0.593891 ( diff = -0.000105 ) train-gini = 0.187573 val-gini = 0.187783
    9 | 01m37s |    0.18778 |             0.6187 |   49.5867 |           0.3332 |      2.4785 |             0.1047 |      0.6622 | 
 Stopped after 500 iterations with train-auc

In [106]:
xgb1=xgb.XGBClassifier()
pred=xgb1.fit(X_train,y_train).predict_proba(X_valid)
pred1 = xgb1.predict_proba(test_cat2)
print(log_loss(y_valid,pred[:,1]))
t2=time.time()
t2-t1

0.5896495644698072


158.776305437088

In [107]:
X_train, X_valid, y_train, y_valid = train_test_split(train_cat3,train_y,test_size=0.33, random_state=42)

In [108]:
xgb1=xgb.XGBClassifier()
pred=xgb1.fit(X_train,y_train).predict_proba(X_valid)
pred2 = xgb1.predict_proba(test_cat3)
print(log_loss(y_valid,pred[:,1]))
t2=time.time()
t2-t1

0.577496717306822


188.350280046463

In [81]:
t1=time.time()
import xgboost as xgb
import warnings
print('-'*130)
log_file = open('Porto-AUC-5fold-XGB-run-01-v1-full1.log', 'a')
AUCbest = -1.
ITERbest = 0
y_dtrain=pd.get_dummies(y_train)
dtrain = xgb.DMatrix(X_train, label = y_dtrain['Y'])
print('-'*130, file=log_file)
log_file.flush()

with warnings.catch_warnings():
    warnings.filterwarnings('ignore')
    XGB_BO.maximize(init_points=2, n_iter=5, acq='ei', xi=0.0)
    
model.append(XGB_BO.res['max']['max_params'])

m_params = model[2]
xgb1=xgb.XGBClassifier(max_depth=int(m_params['max_depth']),gamma=m_params['gamma'],colsample_bytree=m_params['colsample_bytree'],
                       eval_metric='logloss',max_delta_step= m_params['max_delta_step'],min_child_weight=m_params['min_child_weight'],
                      subsample= m_params['subsample'])
pred=xgb1.fit(X_train,y_train).predict_proba(X_valid)
pred2 = xgb1.predict_proba(test_cat3)
print(log_loss(y_valid,pred[:,1]))

t2=time.time()
print(t2-t1)

----------------------------------------------------------------------------------------------------------------------------------
Bayesian Optimization
----------------------------------------------------------------------------------------------------------------------------------
 Step |   Time |      Value |   colsample_bytree |     gamma |   max_delta_step |   max_depth |   min_child_weight |   subsample | 
 Stopped after 500 iterations with train-auc = 0.595224 val-auc = 0.595440 ( diff = -0.000216 ) train-gini = 0.190447 val-gini = 0.190880
   13 | 01m35s |    0.19088 |             0.7781 |   49.1454 |           0.0877 |      2.3755 |            19.6348 |      0.5793 | 
 Stopped after 500 iterations with train-auc = 0.595876 val-auc = 0.596075 ( diff = -0.000198 ) train-gini = 0.191753 val-gini = 0.192150
   14 | 01m46s |    0.19215 |             0.8981 |   49.8242 |           0.6018 |      2.0148 |            18.9584 |      0.4730 | 
 Stopped after 500 iterations with train-auc

In [109]:
X_train, X_valid, y_train, y_valid = train_test_split(train_cat4,train_y,test_size=0.33, random_state=42)

In [110]:
xgb1=xgb.XGBClassifier()
pred=xgb1.fit(X_train,y_train).predict_proba(X_valid)
pred3 = xgb1.predict_proba(test_cat4)
print(log_loss(y_valid,pred[:,1]))
t2=time.time()
t2-t1

0.5661399697647229


251.78417444229126

In [82]:
t1=time.time()
import xgboost as xgb
import warnings
print('-'*130)
log_file = open('Porto-AUC-5fold-XGB-run-01-v1-full1.log', 'a')
AUCbest = -1.
ITERbest = 0
y_dtrain=pd.get_dummies(y_train)
dtrain = xgb.DMatrix(X_train, label = y_dtrain['Y'])
print('-'*130, file=log_file)
log_file.flush()

with warnings.catch_warnings():
    warnings.filterwarnings('ignore')
    XGB_BO.maximize(init_points=2, n_iter=5, acq='ei', xi=0.0)
    
model.append(XGB_BO.res['max']['max_params'])

m_params = model[3]
xgb1=xgb.XGBClassifier(max_depth=int(m_params['max_depth']),gamma=m_params['gamma'],colsample_bytree=m_params['colsample_bytree'],
                       eval_metric='logloss',max_delta_step= m_params['max_delta_step'],min_child_weight=m_params['min_child_weight'],
                      subsample= m_params['subsample'])
pred=xgb1.fit(X_train,y_train).predict_proba(X_valid)
pred3 = xgb1.predict_proba(test_cat4)
print(log_loss(y_valid,pred[:,1]))

t2=time.time()
print(t2-t1)

----------------------------------------------------------------------------------------------------------------------------------
Bayesian Optimization
----------------------------------------------------------------------------------------------------------------------------------
 Step |   Time |      Value |   colsample_bytree |     gamma |   max_delta_step |   max_depth |   min_child_weight |   subsample | 
 Stopped after 500 iterations with train-auc = 0.586948 val-auc = 0.587258 ( diff = -0.000311 ) train-gini = 0.173895 val-gini = 0.174517
   18 | 01m13s |    0.17452 |             0.4084 |   19.2594 |           0.1118 |      2.2326 |             0.4105 |      0.4886 | 
 Stopped after 500 iterations with train-auc = 0.588576 val-auc = 0.588795 ( diff = -0.000220 ) train-gini = 0.177151 val-gini = 0.177590
   19 | 01m32s |    0.17759 |             0.6855 |   45.4355 |           0.5617 |      2.0154 |            19.4422 |      0.4436 | 
 Stopped after 500 iterations with train-auc

In [111]:
pred=pd.DataFrame({'cat1':pred0[:,1],'cat2':pred1[:,1],'cat3':pred2[:,1],'cat4':pred3[:,1]},index=range(0,len(pred0[:,1])))
pred_mean = pred.mean(axis=1)
pred_min = pred.min(axis=1)
pred_max = pred.max(axis=1)

In [96]:
test = pd.read_table('scoring_set.tsv')
columns=[]
for col in test.columns:
    columns.append(col.replace('scoring_set.',''))
test.columns=columns
test_y=test['renewed_yorn']

/usr/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (6,16,17,26,40,41,49,53,54,55,100,104,107,109,112,127,128,129,130,153,171,220,221,222,225,226,227) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [112]:
#submit the prediction score
submit = pd.DataFrame()
submit['INNOVATION_CHALLENGE_KEY'] = test_ids
#RENEWAL_PROBABLIITY
submit['RENEWAL_PROBABLIITY']=pred_mean
submit=submit.sort_values('INNOVATION_CHALLENGE_KEY')
submit.to_csv('XGB-default-model_imputed_mean.csv',index=False)

In [99]:
#submit the prediction score
submit = pd.DataFrame()
submit['INNOVATION_CHALLENGE_KEY'] = test_ids
#RENEWAL_PROBABLIITY
submit['RENEWAL_PROBABLIITY']=pred_min
submit=submit.sort_values('INNOVATION_CHALLENGE_KEY')
submit.to_csv('XGB-BO-model_imputed_min.csv',index=False)

In [100]:
#submit the prediction score
submit = pd.DataFrame()
submit['INNOVATION_CHALLENGE_KEY'] = test_ids
#RENEWAL_PROBABLIITY
submit['RENEWAL_PROBABLIITY']=pred_max
submit=submit.sort_values('INNOVATION_CHALLENGE_KEY')
submit.to_csv('XGB-BO-model_imputed_max.csv',index=False)